## Training Data

#### TO-DO: Eliminate using lists and export generated training data for repeatability

In [ ]:
import matplotlib.pyplot as plt

from mkidreadoutanalysis.resonator import *
import numpy as np
import numpy
import torch
import math

In [ ]:
def gen_iq(qp_timestream: QuasiparticleTimeStream):
    '''
    Generate I and Q time streams using the mkidreadoutanalysis library

    Inputs: 
        QuasiPartcileTimeStream object with populated photons
    '''

    #Creating a resonator object
    resonator = Resonator(f0=4.0012e9, qi=200000, qc=15000, xa=1e-9, a=0, tls_scale=1e2)
    rf = RFElectronics(gain=(3.0, 0, 0), phase_delay=0, cable_delay=50e-9)
    freq = FrequencyGrid( fc=4.0012e9, points=1000, span=500e6)
    sweep = ResonatorSweep(resonator, freq, rf)

    #Creating Photon Resonator Readout
    lit_res_measurment = ReadoutPhotonResonator(resonator, qp_timestream, freq, rf)
    lit_res_measurment.noise_on = True #toggle white noise and line noise
    lit_res_measurment.rf.noise_scale = 10 #adjust white noise scale

    #configure line noise for Resonator
    lit_res_measurment.rf.line_noise.freqs = ([60, 50e3, 100e3, 250e3, -300e3, 300e3, 500e3]) # Hz and relative to center of bin (MKID we are reading out)
    lit_res_measurment.rf.line_noise.amplitudes = ([0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.01])
    lit_res_measurment.rf.line_noise.phases = ([0, 0.5, 0,1.3,0.5, 0.2, 2.4])

    #Generating Synthetic Data for Output
    I = lit_res_measurment.normalized_iq.real
    Q = lit_res_measurment.normalized_iq.imag

    return I, Q 

In [ ]:
def create_windows(i: numpy.array,
                   q: numpy.array,
                   photon_arrivals: numpy.array,
                   with_pulses: list,
                   no_pulses: list,
                   num_samples: int,
                   no_pulse_fraction: float,
                   window_size=150):
    """
    This function takes the output of the mkidreadoutanalysis objects (I, Q, and Photon Arrival vectors) and chunks it into smaller arrays. The code
    also separates chunks with photons and those without photons with the goal of limiting the number of samples
    without photon pulses since there are vastly more windows in the synthetic data in this category. It uses "scanning" logic to scan over the full
    arrays with a given window size and inspects that window for a photon event. The window is then added to the appropriate container (photon/no photon).
    """

    # First determine the last index in the scanning range (need to have length of photon arrivals array be multiple of window_size)
    end_index = math.floor(len(photon_arrivals) / window_size) * window_size

    # Determine all the indicies in the i,q, photon_arrvial vecs denoting a pulse
    pulse_indices = np.argwhere(photon_arrivals == 1).squeeze()

    # Now scan across the photon arrival vector and look at windows of length window_size with and without photon events
    for window in range(0, end_index - window_size + 1, window_size):
        # Check to see if any of the pulse indices are in this window
        if np.sum((pulse_indices >= window) & (pulse_indices < window + 150)) > 0:
            # If so add the window to the with_pulses container
            with_pulses.append(np.vstack((i[window : window + 150],
                                          q[window : window + 150],
                                          photon_arrivals[window : window + 150])).reshape(3, window_size)) # Reshaping to get in nice form for CNN
        # If no pulses are in the window and the no-pulse fraction hasn't been met,
        # add to the no_pulses container
        elif len(no_pulses) < num_samples * no_pulse_fraction:
            no_pulses.append(np.vstack((i[window : window + 150],
                                        q[window : window + 150],
                                        photon_arrivals[window : window + 150])).reshape(3, window_size)) # Reshaping to get in nice form for CNN
 
    
        # Continue to next window doing nothing since threshold for no_pulse samples has been met and the window lacks a pulse
        else:
            continue

In [ ]:
# Now that we know the windowing function works, lets build a dataset

NO_PULSE_FRACTION = 0.3
NUM_SAMPLES = 20000 # This is approximate, the number of photons in the last iteration of the loop is Poisson distributied
QP_TIME_LENGTH = 0.01 # secs
SAMPLING_FREQ = 2e6 # Hz
WINDOW_SIZE = 150
RANDOM_SEED = 42
no_pulses = []
pulses = []
 
# Generating Quasiparticle Timestream
quasiparticle_timestream = QuasiparticleTimeStream(SAMPLING_FREQ, QP_TIME_LENGTH, seed=RANDOM_SEED)
quasiparticle_timestream.gen_quasiparticle_pulse(tf = 30)

# Generate the training set in the following format: [np.array([i,q,label]), ...] where i,q,label are all WINDOW_SIZE length arrays.
# Each list element is a 3 x 1 x WINDOW_SIZE numpy array.
#print(f'Pulse samples: {NUM_SAMPLES - (NUM_SAMPLES * NO_PULSE_FRACTION)}')
#print(f'Noise samples: {NUM_SAMPLES * NO_PULSE_FRACTION}')
while len(pulses) < NUM_SAMPLES - (NUM_SAMPLES * NO_PULSE_FRACTION):
    # We want the training data to be varied, so lets use the Poisson sampled
    # gen_photon_arrivals method to change the photon flux per iteration
    photon_arrivals = quasiparticle_timestream.gen_photon_arrivals(cps=2000)
    quasiparticle_timestream.populate_photons() # this is necessary for the resonator object in the gen_iq function
    I, Q = gen_iq(quasiparticle_timestream)
    create_windows(I, Q, photon_arrivals, pulses, no_pulses, NUM_SAMPLES, NO_PULSE_FRACTION, window_size=WINDOW_SIZE)
    print(f'Number of samples with pulses so far: {len(pulses)}')
    print(f'Number of samples without pulses so far: {len(no_pulses)}')

In [ ]:
## Now lets inspect some of the windows with pulses
#import time
#t = np.arange(0, WINDOW_SIZE)
#for ind in range(len(pulses)):
#    fig, ax = plt.subplots(3,1,figsize = (18,18))
#    ax[0].plot(t, pulses[ind][0])
#    ax[0].set_xlabel("Time (us)")
#    ax[0].set_ylabel("I Trace",fontweight = "bold", size = 'large')
#
#    ax[1].plot(t, pulses[ind][1])
#    ax[1].set_xlabel("Time (us)")
#    ax[1].set_ylabel("Q Trace",fontweight = "bold", size = 'large')
#
#    ax[2].plot(t, pulses[ind][2])
#    ax[2].set_xlabel("Time (us)")
#    ax[2].set_ylabel("Photon Arrivals (with pulses)",fontweight = "bold", size = 'large')
#    plt.show()
#    time.sleep(5)

In [ ]:
## Now lets inspect some of the windows without pulses
#for ind in range(len(pulses)):
#    fig, ax = plt.subplots(3,1,figsize = (18,18))
#    ax[0].plot(t, no_pulses[ind]['i'])
#    ax[0].set_xlabel("Time (us)")
#    ax[0].set_ylabel("I Trace",fontweight = "bold", size = 'large')
#
#    ax[1].plot(t, no_pulses[ind]['q'])
#    ax[1].set_xlabel("Time (us)")
#    ax[1].set_ylabel("Q Trace",fontweight = "bold", size = 'large')
#
#    ax[2].plot(t, no_pulses[ind]['label'])
#    ax[2].set_xlabel("Time (us)")
#    ax[2].set_ylabel("Photon Arrivals (without pulses)",fontweight = "bold", size = 'large')
#    plt.show()
#    time.sleep(5)

In [ ]:
from random import shuffle

X = []
y = []

# Lets create one big list of the pulse and no pulse samples randomly shuffled together 
train_data = pulses + no_pulses
shuffle(train_data)

# Now lets separate the training samples (I/Q data) from the label data (photon arrival)
for element in train_data:
    X.append(element[0:2,:])
    y.append(element[2].reshape(1,WINDOW_SIZE))

In [ ]:
# With the training and label data now separated, lets start defining our training/testing metrics
# and split the dataset into train and test
TEST_RATIO = 0.2
BATCH_SIZE = 32

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=TEST_RATIO, # Ratio of test data to use from full dataset; Training is the complement
    random_state=RANDOM_SEED
)
print(f'# of train samples: {len(X_train)}, # of test samples: {len(X_test)}')

In [ ]:
# Lets visualize a few training samples and the associated label to make sure things are looking good before creating the
# Dataloader objects
import time
#t = np.arange(0, WINDOW_SIZE)
#for count in range(10):
#    fig, ax = plt.subplots(3,1,figsize = (18,18))
#    # Choose a random sample from the training set
#    ind = torch.randint(0, len(X_train) - 1, size=[1]).item()
#    ax[0].plot(t, X_train[ind][0].squeeze())
#    ax[0].set_xlabel("Time (us)")
#    ax[0].set_ylabel("I Trace",fontweight = "bold", size = 'small')
#
#    ax[1].plot(t, X_train[ind][1].squeeze())
#    ax[1].set_xlabel("Time (us)")
#    ax[1].set_ylabel("Q Trace",fontweight = "bold", size = 'small')
#
#    ax[2].plot(t, y_train[ind][0].squeeze())
#    ax[2].set_xlabel("Time (us)")
#    ax[2].set_ylabel("Photon Arrivals",fontweight = "bold", size = 'small')
#    plt.show()
#    time.sleep(5)

In [ ]:
# It's finally time to create our Dataloader objects
from torch.utils.data import TensorDataset, DataLoader

# Let's first convert from numpy arrays to Tensors
train_dataset = TensorDataset(torch.Tensor(numpy.array(X_train)),
                              torch.Tensor(numpy.array(y_train)))
test_dataset = TensorDataset(torch.Tensor(numpy.array(X_test)),
                             torch.Tensor(numpy.array(y_test)))

train_dloader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)
test_dloader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False
)

# Now lets inpect the objects. Much of the logic to batch and match labels to input data is done for us in the Dataloader obj
print(f'Type: {type(train_dloader)}')
train_batch_img, train_batch_labels = next(iter(train_dloader))
print(f'Batch Img: {train_batch_img.shape}, Batch Labels: {train_batch_labels.shape}') # So the dloader is basically a list of all the batches and their corressponding labels

## Model Definition

#### With the training data ready, lets now define the model and the train/test loop

In [ ]:
from torch import nn
torch.manual_seed(RANDOM_SEED)

class ConvAE(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.encoder = ConvEncoder(in_channels)
        self.decoder = ConvDecoder()

    def forward(self, x):
        return self.decoder(self.encoder(x))

class ConvEncoder(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        """
        This class defines the encoder portion of the Convolution Autoencoder network
        """
        self.block1 = nn.Sequential(
            nn.Conv1d(in_channels=in_channels, out_channels=16, kernel_size=2, stride=1, padding='same'), 
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2) # This is a compression step in this model (1x150 -> 1x75)
        )
        self.block2 = nn.Sequential(
            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding='same'), 
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=3) # The second compression step (1x75 -> 1x25)
        )
        self.block3 = nn.Sequential(
            nn.Conv1d(in_channels=32, out_channels=128, kernel_size=5, stride=1, padding='same'),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=5) # The second compression step (1x25 -> 1x5)
        )
    
    def forward(self, x):
        z = self.block3(self.block2(self.block1(x)))
        return z

class ConvDecoder(nn.Module):
    def __init__(self):
        super().__init__()
        """
        This class defines the decoder portion of the Convolution Autoencoder network
        """
        self.block1 = nn.Sequential(
            nn.Conv1d(in_channels=128, out_channels=32, kernel_size=1, stride=1, padding='same'), 
            nn.ReLU(),
            nn.Upsample(scale_factor=5, mode='linear') # First decompression step (1x5 -> 1x25)
        )
        self.block2 = nn.Sequential(
            nn.Conv1d(in_channels=32, out_channels=16, kernel_size=2, stride=1, padding='same'), 
            nn.ReLU(),
            nn.Upsample(scale_factor=3, mode='linear') # second decompression step (1x25 -> 1x75)
        )
        self.block3 = nn.Sequential(
            nn.Conv1d(in_channels=16, out_channels=1, kernel_size=2, stride=1, padding='same'), 
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='linear'), # third decompression step (1x75 -> 1x150)
            nn.Sigmoid() # normalize values
        )

    def forward(self, x):
        z = self.block3(self.block2(self.block1(x)))
        return z

In [ ]:
# Now we can create the train/test loop functions
from tqdm.auto import tqdm
from timeit import default_timer as timer

def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn):

    # Training Steps
    loss, acc = 0, 0 # need to reset loss every epoch
    for batch, (X, y) in enumerate(data_loader): # each batch has 32 data/labels, create object -> (batch, (X, y))
        model.train()
        y_pred = model(X) # Like before, need to get model's predictions
        loss = loss_fn(y_pred, y) # calculate loss for this batch
        loss += loss # add loss from this batch (mean loss of 32 samples) to total loss for the epoch (sum of all batch loss)
        acc += accuracy_fn(y_pred, y)

        # backprop step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # We want to see some updates within an epoch
        print(f'Batches processed: {batch + 1}/{len(data_loader)}, Samples processed: {(batch + 1) * data_loader.batch_size}/{len(data_loader.dataset)}', end='\r')
    
    # Now we want to find the AVERAGE loss and accuracy of all the batches
    loss /= len(data_loader)
    acc /= len(data_loader)
    print('\n-----------')
    print(f'Mean Train Loss: {loss:.4f}, Mean Train Accuracy: {acc * 100:.2f}%')

def test_step(model: torch.nn.Module,
              data_loader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              accuracy_fn):

    # Test Steps
    model.eval()
    loss, acc = 0, 0 # need to reset loss every epoch
    with torch.inference_mode():
        for X, y in data_loader: # each batch has 32 data/labels, create object -> (batch, (X_train, y_train))
            y_pred = model(X) # Like before, need to get model's predictions
            loss = loss_fn(y_pred, y) # calculate loss for this batch
            loss += loss # add loss from this batch (mean loss of 32 samples) to total loss for the epoch (sum of all batch loss)
            acc += accuracy_fn(y_pred, y)

        # Now we want to find the AVERAGE loss and accuracy of all the batches
        loss /= len(data_loader)
        acc /= len(data_loader)
    print(f'Mean Test Loss: {loss:.4f}, Mean Test Accuracy: {acc * 100:.2f}%')
    print('-----------\n')

def accuracy_fn(y_pred, y_true):
    """Calculates accuracy between truth labels and predictions.

    Args:
        y_true (torch.Tensor): Truth labels for predictions.
        y_pred (torch.Tensor): Predictions to be compared to predictions.

    Returns:
        [torch.float]: Accuracy value between y_true and y_pred, e.g. 78.45
    """
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred))
    return acc



In [ ]:
# Now lets define the model, loss function, and optimzer
convae_model_v1 = ConvAE(in_channels=2)
optimizer = torch.optim.SGD(params=convae_model_v1.parameters(), lr=0.3)
loss_fn = nn.L1Loss(reduction='mean')# 'mean' reduction takes all the loss values from the batch and averages them to get the loss
convae_model_v1

In [ ]:
# Now lets give the train/test loop a try!

# Now lets create a quick little function that gives the run time of the loop
total_time = lambda start_time, stop_time: stop_time - start_time

EPOCHS = 20
train_time_cnn_start = timer()
for epoch in tqdm(range(EPOCHS)):
    print(f'Epoch: {epoch}\n-----------')
    train_step(
        convae_model_v1,
        train_dloader,
        loss_fn,
        optimizer,
        accuracy_fn
    )
    test_step(
        convae_model_v1,
        test_dloader,
        loss_fn,
        accuracy_fn
    )
train_time_cnn_end = timer()
print(f'Total time to train: {total_time(train_time_cnn_start, train_time_cnn_end):.2f}s')

In [ ]:
def make_predictions(model: nn.Module, samples: list):
    """
    Given a list of samples, returns a list of the prediction for each sample
    """
    model.eval()
    with torch.inference_mode():
        preds_list = [model(x) for x in tqdm(samples, desc="Making predictions...")] 
        return torch.cat(preds_list) # return as tensor instead of list

# Pick n random samples/labels from the test data
test_samples = []
test_labels = []

import random
for sample, label in random.sample(list(test_dataset), k=10): # random.sample samples k elements from the given population without replacement; returns list of samples.
    test_samples.append(sample)
    test_labels.append(label)

print(f'Test Sample Shape: {test_samples[0].shape}, Test Label Shape: {test_labels[0].shape}')
preds = make_predictions(convae_model_v1, [x.unsqueeze(dim=0) for x in test_samples]) 


In [ ]:
# Lets see if the model works at all..
t = torch.arange(0, 150).squeeze()
import time
for i in range(len(test_samples)):
    fig, ax = plt.subplots(4,1,figsize = (18,18))
    ax[0].plot(t, test_samples[i][0])
    ax[0].set_xlabel("Time (us)")
    ax[0].set_ylabel("I Trace",fontweight = "bold", size = 'large')
    
    ax[1].plot(t, test_samples[i][1])
    ax[1].set_xlabel("Time (us)")
    ax[1].set_ylabel("Q Trace",fontweight = "bold", size = 'large')
    
    ax[2].plot(t, test_labels[i].squeeze())
    ax[2].set_xlabel("Time (us)")
    ax[2].set_ylabel("Photon Arrivals",fontweight = "bold", size = 'large')
    
    ax[3].plot(t, preds[i].squeeze())
    ax[3].set_xlabel("Time (us)")
    ax[3].set_ylabel("Photon Arrivals Prediction)",fontweight = "bold", size = 'large')
    plt.show()
    time.sleep(5)

In [ ]:
# The model doesn't seem to be working, but I think it's a machinery issue, not a design issue. Lets try to simplify the model to use a basic autoencoder (no convolutions).